In [1]:
pip install torchvision==0.1.9

  Using cached torchvision-0.1.9-py2.py3-none-any.whl (43 kB)
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.13.1
    Uninstalling torchvision-0.13.1:
      Successfully uninstalled torchvision-0.13.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
from cv2 import imshow as cv2_imshow
import numpy
import math
import numpy as np
import matplotlib.pyplot as plt
import torchvision
import os

In [3]:
def otsu(img):
  img = np.array(img)
  pxls = img.flatten()
  img_dict = {}
  for i in sorted(pxls):
    if i not in img_dict:
      img_dict[i] = 1
    else:
      img_dict[i] +=1

  N = 0
  for i in img_dict.values():
    N += i
  density = {}
  for i in img_dict.keys():
    density[i] = img_dict[i]/N
  mean_l = {}
  weight_l = {}
  sum = 0
  w = 0
  for i in range(len(density)):
    w += density[list(density.keys())[i]]
    
    weight_l[list(density.keys())[i]] = w
    sum += density[list(density.keys())[i]] * np.sum(pxls)
    numer = 0
    for j in range(i + 1):
      numer += density[list(density.keys())[j]]* np.sum(pxls) * list(density.keys())[j]
    mean_l[list(density.keys())[i]] = numer/sum
  mean_r = {}
  dens_l = [(i, density[i]) for i in density]
  for i in range(len(dens_l)):
    numer = 0
    sum = 0
    if(i < len(dens_l) - 1):
      for j in range(i + 1, len(dens_l)):
        numer += dens_l[j][0]*dens_l[j][1]*np.sum(pxls)
        sum += dens_l[j][1]*np.sum(pxls)
    if(dens_l[i][0] != dens_l[len(dens_l)-1][0]):
      mean_r[dens_l[i][0]] = numer/sum
    else:
      mean_r[dens_l[i][0]] = 0
  vars = {}
  for i in mean_l:
    var = (w-weight_l[i])*weight_l[i]*((mean_l[i] - mean_r[i])**2)
    vars[i] = var
  threshold = sorted(vars, key = vars.get)[-1]
  for i in range(len(img)):
    for j in range(len(img[i])):
      if(img[i][j] <= threshold):
        img[i][j] = 0
      else:
        img[i][j] = 255
  return img

In [4]:
signum = lambda x: -1 if x < 0 else 1

cells = [(-1, -1), (-1, 0), (-1, 1), (0, 1), (1, 1), (1, 0), (1, -1), (0, -1), (-1, -1)]

def get_angle(left, right):
    angle = left - right
    if abs(angle) > 180:
        angle = -1 * signum(angle) * (360 - abs(angle))
    return angle

In [5]:
def poincare_index_at(i, j, angles, tolerance):
    deg_angles = [math.degrees(angles[i - k][j - l]) % 180 for k, l in cells]
    index = 0
    for k in range(0, 8):
        if abs(get_angle(deg_angles[k], deg_angles[k + 1])) > 90:
            deg_angles[k + 1] += 180
        index += get_angle(deg_angles[k], deg_angles[k + 1])

    if 180 - tolerance <= index and index <= 180 + tolerance:
        return "loop"
    if -180 - tolerance <= index and index <= -180 + tolerance:
        return "delta"
    if 360 - tolerance <= index and index <= 360 + tolerance:
        return "whorl"
    return "none"

In [6]:
def calculate_singularities(im, angles, tolerance, W):
    (y, x) = im.shape
    
    result = cv2.cvtColor(im,cv2.COLOR_GRAY2RGB)
    im = cv2.cvtColor(im, cv2.COLOR_GRAY2RGB)
    colors = {"loop" : (255, 0, 0), "delta" : (0, 255, 0), "whorl": (0, 0, 255)}
    positions = {"loop" : [], "delta" : [], "whorl": [], "none": []}
    for i in range(1, len(angles) - 1):
        for j in range(1, len(angles[i]) - 1):
            singularity = poincare_index_at(i, j, angles, tolerance)
            if singularity != "none":
                kernel = get_kernel(im, (i*W, j*W))
                draw_elipse(result, (i*W, j*W), colors[singularity])
                positions[singularity].append(((i * W, j * W), kernel))
            else:
                kernel = get_kernel(im, (i*W, j*W))
                if(len(positions[singularity])<8):
                    positions[singularity].append(((i * W, j * W), kernel))

    return result, positions

In [7]:
def get_kernel(image, center):
    (x, y) = center
    
    kernel = []
    for i in range(max(0, x-8), min(x+8, 239)):
        temp = []
        for j in range(max(0, y - 8), min(y+8, 319)):
            temp.append(list(image[i][j]))
        kernel.append(temp)
    return kernel

In [8]:
def apply_kernel_at(get_value, kernel, i, j):
    kernel_size = 3
    result = 0
    for k in range(0, kernel_size):
        for l in range(0, kernel_size):
            pixel = get_value(i + k - kernel_size / 2, j + l - kernel_size / 2)
            result += pixel * kernel[k][l]
    return result

In [9]:
def transpose(ls):
    return map(list, zip(*ls))

In [10]:
def calculate_angles(im, W, f, g):
    (y, x) = im.shape
    im_load = im
    get_pixel = lambda x, y: im_load[int(y)][int(x)]
    
    sobelOperator = [[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]]
    
    ySobel = list(sobelOperator)
    xSobel = list(transpose(sobelOperator))

    result = [[] for i in range(1, x, W)]
    G = [[] for i in range(1, x, W)]

    for i in range(1, x, W):
        for j in range(1, y, W):
            nominator = 0
            denominator = 0
            for k in range(i, min(i + W , x - 1)):
                for l in range(j, min(j + W, y - 1)):
                    Gx = apply_kernel_at(get_pixel, xSobel, k, l)
                    Gy = apply_kernel_at(get_pixel, ySobel, k, l)

                    nominator += f(Gx, Gy)
                    denominator += g(Gx, Gy)
            angle = (math.pi + math.atan2(nominator, denominator)) / 2
            result[(i - 1) // W].append(angle)

    return result

In [11]:
def draw_elipse(image, center, color):
  (x, y) = center
  cv2.ellipse(image, (x, y), (16, 16), 0, 0, 360, color, 2)

In [12]:
def poincare_indices(url):
  img = cv2.imread(url, cv2.COLOR_BGR2GRAY)
  blockSize = 32
  f = lambda x, y: 2 * x * y
  g = lambda x, y: x ** 2 - y ** 2
  img = otsu(img)

  angles = calculate_angles(img, blockSize, f, g)
  img_t = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
  colors = {"loop" : (255, 0, 0), "delta" : (0, 255, 0), "whorl": (0, 0, 255)}
  result, positions = calculate_singularities(img, angles, 1, blockSize)
    
  return positions 


In [ ]:
path = r'C:\Users\Arsalan\Desktop\Research\Fingerprints'
pics = os.listdir(path)
count = 0
for img in pics:
    img_p = path+ "\\" + img
    try:
        positions = poincare_indices(img_p)
        add_to_dataset(img, positions, count)
        count += 1
    except:
        print(img)


In [96]:
def add_to_dataset(img, positions, count):
    path_pores = r'C:\Users\Arsalan\Desktop\Research\Pores'
    with open('Pores.csv', 'a') as file1:
        i = 0
        sing_int = 0
        for singularity in positions:
            for each in positions[singularity]:
                path_res = os.path.join(path_pores, img[:-4] + '_' + str(i) + '.jpg')
                cv2.imwrite(path_res, numpy.array(each[1]))
                file1.write(path_res + ", " + str(sing_int) + '\n')
                i+=1
            sing_int+=1
        

In [92]:
pores = [[], [], [], []]
with open("Pores.csv", 'r') as file1:
    while(1==1):
        line = file1.readline()
        if not line:
            break
        path, singularity = line.split()
        path = path[:-1]
        singularity = int(singularity)
        pores[singularity].append(path)

In [93]:
for i in pores:
    print(len(i), end=' ')

849 2728 231 1288 

In [86]:
with open("Pores.csv", 'r') as file1:
    while(1==1):
        line = file1.readline()
        if not line:
            break
        path, singularity = line.split()
        path = path[:-1]
        singularity == int(singularity)
        if(singularity == '2'):
            if path[-7:-4] == "_sp" or path[-7:-4] == "_sh" or path[-7:-4] == '_bl':
                if os.path.isfile(path):
                    print(path)
                    os.remove(path)

In [6]:

img = cv2.imread(r'C:\Users\Arsalan\Desktop\Research\Fingerprints\6_1_1.jpg')
mir_img = vert_mirror(img)
cv2.imshow('', img)
cv2.waitKey()
cv2.destroyAllWindows()
print(img.shape)

(240, 320, 3)


In [14]:
print(pores[1][0][:-4])

C:\Users\Arsalan\Desktop\Research\Pores\118_2_2_4


In [90]:
with open("Pores.csv", 'a') as file1:
    for i in pores[2]:

        img = cv2.imread(i)
        path = i[:-4]

        img = cv2.imread(i)
        path = i[:-4]
        
        Sepia_Kernel = np.array([[0.272, 0.534, 0.131],[0.349, 0.686, 0.168],[0.393, 0.769, 0.189]])
        Sharpen_Kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
        Emboss_Kernel = np.array([[0,-1,-1],[1,0,-1],[1,1,0]])

        Sepia_Effect_Img = cv2.filter2D(src=img, kernel=Sepia_Kernel, ddepth=-1)
        Sharpen_Effect_Img = cv2.filter2D(src=img, kernel=Sharpen_Kernel, ddepth=-1)
        Sharpen_Sepia = cv2.filter2D(src=Sharpen_Effect_Img, kernel=Sepia_Kernel, ddepth=-1)
        Blur_Effect_Img = cv2.GaussianBlur(img, (5, 5), 0)
        Blur_Sepia = cv2.GaussianBlur(Sepia_Effect_Img, (5,5), 0)


        cv2.imwrite(os.path.join(path + '_' + 'sp' + '.jpg'), Sepia_Effect_Img)
        file1.write(os.path.join(path + '_' + 'sp' + '.jpg') + ", " + str(2) + '\n')
        
        cv2.imwrite(os.path.join(path + '_' + 'sh' + '.jpg'), Sharpen_Effect_Img)
        file1.write(os.path.join(path + '_' + 'sh' + '.jpg') + ", " + str(2) + '\n')
        
        cv2.imwrite(os.path.join(path + '_' + 'sh' + '_' + 'sp' + '.jpg'), Sharpen_Sepia)
        file1.write(os.path.join(path + '_' + 'sh' + '_' + 'sp' '.jpg') + ", " + str(2) + '\n')
        #h_v
        cv2.imwrite(os.path.join(path + '_' + 'bl' + '.jpg'), Blur_Effect_Img)
        file1.write(os.path.join(path + '_' + 'bl'+ '.jpg') + ", " + str(2) + '\n')

        cv2.imwrite(os.path.join(path + '_sp' + '_bl' + '.jpg'), Blur_Sepia)
        file1.write(os.path.join(path + '_sp' + '_bl'+ '.jpg') + ", " + str(2) + '\n')

In [89]:
with open("Pores.csv", 'a') as file1:
    for i in pores[1]:

        img = cv2.imread(i)
        path = i[:-4]
        
        Sepia_Kernel = np.array([[0.272, 0.534, 0.131],[0.349, 0.686, 0.168],[0.393, 0.769, 0.189]])
        Sharpen_Kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
        Emboss_Kernel = np.array([[0,-1,-1],[1,0,-1],[1,1,0]])

        Sepia_Effect_Img = cv2.filter2D(src=img, kernel=Sepia_Kernel, ddepth=-1)
        Sharpen_Effect_Img = cv2.filter2D(src=img, kernel=Sharpen_Kernel, ddepth=-1)
        Sharpen_Sepia = cv2.filter2D(src=Sharpen_Effect_Img, kernel=Sepia_Kernel, ddepth=-1)
        Blur_Effect_Img = cv2.GaussianBlur(img, (5, 5), 0)
        Blur_Sepia = cv2.GaussianBlur(Sepia_Effect_Img, (5,5), 0)


        cv2.imwrite(os.path.join(path + '_' + 'sp' + '.jpg'), Sepia_Effect_Img)
        file1.write(os.path.join(path + '_' + 'sp' + '.jpg') + ", " + str(1) + '\n')
        
        cv2.imwrite(os.path.join(path + '_' + 'sh' + '.jpg'), Sharpen_Effect_Img)
        file1.write(os.path.join(path + '_' + 'sh' + '.jpg') + ", " + str(1) + '\n')
        
        cv2.imwrite(os.path.join(path + '_' + 'sh' + '_' + 'sp' + '.jpg'), Sharpen_Sepia)
        file1.write(os.path.join(path + '_' + 'sh' + '_' + 'sp' '.jpg') + ", " + str(1) + '\n')
        #h_v
        cv2.imwrite(os.path.join(path + '_' + 'bl' + '.jpg'), Blur_Effect_Img)
        file1.write(os.path.join(path + '_' + 'bl'+ '.jpg') + ", " + str(1) + '\n')

        cv2.imwrite(os.path.join(path + '_sp' + '_bl' + '.jpg'), Blur_Sepia)
        file1.write(os.path.join(path + '_sp' + '_bl'+ '.jpg') + ", " + str(1) + '\n')

        


In [4]:
def vert_mirror(img):
    img = numpy.ndarray.tolist(img)
    for i in range(len(img)):
        for j in range(len(img[i])//2):
            img[i][j], img[i][len(img[i]) - 1 - j] = img[i][len(img[i]) - 1 - j], img[i][j]
    return numpy.array(img)

In [28]:
def hor_mirror(img):
    img = numpy.ndarray.tolist(img)
    for i in range(len(img[0])):
        for j in range(len(img)//2):
            img[j][i], img[len(img) - 1 - j][i] = img[len(img) - 1 - j][i], img[j][i]
    return numpy.array(img)

In [88]:
pics = os.listdir(r'C:\Users\Arsalan\Desktop\Research\Pores')
for i in pics:
    if i[-7:-4] == "_bl" or i[-7:-4] == "_sp" or i[-7:-4] == '_sh':
        print(i)
print(len(pics))

2269


In [68]:
os.remove(r'C:\Users\Arsalan\Desktop\Research\Pores\118_2_2_4_sp.jpg')
os.remove(r'C:\Users\Arsalan\Desktop\Research\Pores\118_2_2_4_sh.jpg')

In [2]:
imgs = os.listdir(r'C:\Users\Arsalan\Desktop\Research\Fingerprints')
print(len(imgs))

210
